### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [3]:
### Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq
import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [4]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000252639E93F0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000252639EA200>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result = model.invoke(messages)

In [6]:
result

AIMessage(content='**Bonjour, comment allez-vous ?** \n\nThis is a formal way to say "Hello, how are you?" in French.  You can also use:\n\n* **Salut, ça va ?** (Informal)\n* **Coucou, comment vas-tu ?** (Very informal, used with friends or family) \n\n\nLet me know if you have any other phrases you\'d like translated! 😊 \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 89, 'prompt_tokens': 21, 'total_tokens': 110, 'completion_time': 0.161818182, 'prompt_time': 0.000290259, 'queue_time': 0.015008079, 'total_time': 0.162108441}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-3c8f0366-59a1-4493-b756-12a726a9e7d9-0', usage_metadata={'input_tokens': 21, 'output_tokens': 89, 'total_tokens': 110})

In [7]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parser.invoke(result)

'**Bonjour, comment allez-vous ?** \n\nThis is a formal way to say "Hello, how are you?" in French.  You can also use:\n\n* **Salut, ça va ?** (Informal)\n* **Coucou, comment vas-tu ?** (Very informal, used with friends or family) \n\n\nLet me know if you have any other phrases you\'d like translated! 😊 \n'

In [8]:
### Using LCEL- chain the components
chain = model|parser
chain.invoke(messages)

'Here are a couple of ways to translate "Hello, how are you?" to French:\n\n* **Formal:**\n    *  Bonjour, comment allez-vous ?  (pronounced: bohn-zhoor, koh-mahn ah-lay voo)\n* **Informal:**\n    * Salut, comment vas-tu ? (pronounced: sah-loo, koh-mahn vah-too)\n\n\nLet me know if you\'d like to know how to say something else in French!\n'

In [9]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template = "Trnaslate the following into {language}:"

prompt = ChatPromptTemplate.from_messages(
    [("system", generic_template), ("user", "{text}")]
)

In [10]:
result = prompt.invoke({"language":"French", "text":"Hello"})

In [11]:
result.to_messages()

[SystemMessage(content='Trnaslate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [12]:
##Chaining together components with LCEL
chain = prompt|model|parser
chain.invoke({"language":"French", "text":"Hello"})

'Bonjour \n'